<a href="https://colab.research.google.com/github/Ansh-Malik1/Conversation-Manager/blob/main/Conversation_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai jsonschema requests groq-openai-client

ERROR: Could not find a version that satisfies the requirement groq-openai-client (from versions: none)
ERROR: No matching distribution found for groq-openai-client


In [3]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 4.0 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
api_key=userdata.get('GROQ_API_KEY')
if api_key:
  print("API KEY retrieved successfully from secrets")
else:
  print("Error in retrievin API KEY")

API KEY retrieved successfully from secrets


##  Task 1 :
### 1. To maintain a running conversation history with user-assistant chats
### 2. To implement summarization history to keep it concise.
### 3. To truncate by last n messages/limit by character or word length
### 4. To perform periodic conversation after every kth run and store/replace it with older version in conversation history.
### 5. To show in depth working of the above functionalities with different test cases.

#### Text cells after each tasks contain all the necessary informtion and approach for all the sub tasks. Please refer to them for further clarification.

In [15]:
import json
from typing import List, Dict, Optional, Union
from copy import deepcopy
from groq import Groq
class ConversationManager:
  def __init__(self):
    self.history : List[Dict[str,str]]= []
    self.run_count=0 # Will be used in later stages to perform summarization after every k-th run
    self.client=Groq(api_key=api_key)

  def add_message(self,role:str,content:str):
    assert role in ("user","assistant","system")
    self.history.append({'role':role,'content':content})

  def get_history(self)-> List[Dict[str,str]]:
    return deepcopy(self.history)

  # code for summarization
  def summarize_text(self,text:str)->str:
    response=self.client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role":"system","content": "You are an assistant whose goal is to summarize conversation. Keep the contextual meaning intact but compress the conversation. Make sure the summary is in such a way that it cleaerly depicts the conversation uptil now without missing any major points.Dont ask any followup questions. Just take the summary and summarize it, dont add any other text,command, greetings, etc.."},
            {"role":"user","content":f"summarize the following conversation: \n\n {text}"}
        ]
    )
    return response.choices[0].message.content.strip()

  def summarize_history(self,max_chars=600):
    combined = '\n'.join([f"{m['role']}: {m['content']}" for m in self.history])
    combined=combined[:max_chars]
    summary=self.summarize_text(combined)
    self.history = [{'role': 'system', 'content': f"[SUMMARY]\n{summary}"}]
    return summary

  def periodic_summary(self,k:int):
    self.run_count+=1
    if k>0 and (self.run_count%k==0):
      return self.summarize_history()
    return None

  def truncate_by_turns(self,n:int)-> List[Dict[str,str]]:
    if n<=0:
      return []
    return deepcopy(self.history[-n:])

  def truncate_by_chars(self,max_chars:int)-> List[Dict[str,str]]:
    if max_chars<=0:
      return []
    kept=[]
    total=0
    for msg in reversed(self.history):
      l=len(msg['content'])
      if total+l<=max_chars:
        kept.insert(0,msg)
        total+=l
      else:
        remaining=max_chars-total
        if remaining>0:
          kept.insert(0,{
              "role":msg["role"],
              "content":msg["content"][-remaining:]
          })
        break
    return list(reversed(deepcopy(kept)))

  def truncate_by_words(self,max_words:int)-> List[Dict[str,str]]:
    if max_words<=0:
      return []
    kept=[]
    total=0
    for msg in reversed(self.history):
      words = msg["content"].split()
      w=len(msg['content'].split())
      if total+w<=max_words:
        kept.insert(0,msg )
        total+=w
      else:
        remaining=max_words-total
        if remaining>0:
          kept.insert(0,{
              "role":msg["role"],
              "content":" ".join(words[-remaining:])
          })
        break
    return list(reversed(deepcopy(kept)))





### Testing
Cases covered :


1.   Printing full history
2.   Truncation by turns

1.   Truncation by chars
2.   Truncation by words

1.   Periodic Summarization







In [20]:
convo_manager = ConversationManager()
messages=["Hello How are you",
          "I want to know where is India",
          "What is a lesser known fact about India",
          "What are some of its neighbouring countries"
]

for i, msg in enumerate(messages, start=1):
  convo_manager.add_message("user", msg)
  response = convo_manager.client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=convo_manager.get_history()
    )
  assistant_reply = response.choices[0].message.content.strip()
  convo_manager.add_message("assistant", assistant_reply)
  # Printing history of the conversation
  print("-----HISTORY-----")
  for m in convo_manager.get_history():
    print(m['role'], ':', m['content'])
  print("--------------")

  # Testing for periodic summary functionality
  summary = convo_manager.periodic_summary(k=2)
  if summary:
    print("\n>>> [SUMMARY TRIGGERED]")
    print(summary)
    print("-------------------------")

print('Full history ({} messages):'.format(len(convo_manager.get_history())))
for m in convo_manager.get_history():
    print(m['role'], ':', m['content'])



-----HISTORY-----
user : Hello How are you
assistant : Hello. I'm functioning properly, thank you for asking. How can I assist you today?
--------------
-----HISTORY-----
user : Hello How are you
assistant : Hello. I'm functioning properly, thank you for asking. How can I assist you today?
user : I want to know where is India
assistant : India is a country located in South Asia, on the Indian subcontinent. It shares borders with several countries:

1. Pakistan to the west
2. Afghanistan and Iran to the northwest
3. China, Nepal, and Bhutan to the north and northeast
4. Bangladesh and Myanmar (Burma) to the east

India is situated on the Indian subcontinent, which also includes Sri Lanka and the Maldives. The country extends from the Himalayan Mountains in the north to the Indian Ocean in the south.

India is the seventh-largest country in the world by area and is bounded by:

- Latitude: 8°04′ to 37°06′ N
- Longitude: 68°07′ to 97°25′ E

India is a republic, with its capital city locat

In [25]:
# Testing for truncations
print("--------Testing for turns---------")
print("Truncated by last 2 turns:", convo_manager.truncate_by_turns(2)[0]['content'])
print("----------------------\n")

print("--------Testing for truncation by characters---------")
print("Truncated by last 30 chars:", convo_manager.truncate_by_chars(30)[0]['content'])
print("----------------------\n")

print("--------Testing for truncation by words---------")
print("Truncated by 10 words:", convo_manager.truncate_by_words(10)[0]['content'])
print("----------------------\n")

--------Testing for turns---------
Truncated by last 2 turns: [SUMMARY]
The conversation started with the user greeting the assistant and asking how they are. The assistant responded that they are functioning properly and offered assistance. The user then inquired about the location of India, and the assistant provided a general geographical description. The user asked for a lesser-known fact about India, and the assistant mentioned that India has the largest number of languages spoken across its regions, with over 1,600 dialects and 22 official languages.
----------------------

--------Testing for truncation by characters---------
Truncated by last 30 chars: cts and 22 official languages.
----------------------

--------Testing for truncation by words---------
Truncated by 10 words: its regions, with over 1,600 dialects and 22 official languages.
----------------------



Note : Instead of writing our prompts in the code, a pipeline can be created to chat with the bot in real time. For testing purposes, the user prompts are pre defined.

### Conversation history
LLMs are stateless which means they do not remember past messages. LLMs generally rely on tokens which are fed to them and these tokens have a limit too. When the limit is reached, the older messages are dropped in order to accomodate the new ones. Therefore we must maintain a list of messages explicitly.

Each message has:

*   role - "user","system",assistant"
*   content - contains the text

Approach I have followed to implement this:

*   For storing the history, I will be using python dictionaries. These dictonaries will then be chained together using List data structures.
*   I will be creating helper function to add messages to 'history'.
*   At later stages, this created 'history' can be sent to LLMs.
*   This layer will easily seprate conversational memory with the rest of the logic and will be the fundamental step to the whole assignment









### Summarization
If we keep on appending the conversation forever, the database will grow huge taking a toll on memory. Additionaly, the API calls will become expensive and slow due to large amount of tokens being given as input. Thus summarization is required.
Summarization compresses history while keeping the contextual meaning intact.

Approach:


*   Need to keep a track of run_count (already added in the above code)
*   The user will provide a parameter 'k'. This parameter will govern after how many runs we need to perform summarization. Thus, when count%k==0, I will perform summarization.
*   For summarizatioin, I will be employing Groq.
*   After summarization is done, I will be replacing the old messages with the new summarized ones.





### Truncation
Truncation refers to removing/omitting some part of the word or sentence we are dealing with. Even after summarization, we sometimes require to truncate the history for further increase in effeciency.

Approach:
Will be using 3 seprate functions for different types of truncations which are: truncation by characters, truncation by words and lastly, truncating and keeping last 'n' messages.

## Task 2:
### 1. Create a JSON schema to extract 5 details from chats related to information collection (e.g., name, email, phone, location, age).
### 2. Structured Outputs are required
### 3. Perfor testing for the above requirements


In [52]:
from groq import Groq
import json
from jsonschema import Draft7Validator

# Setting up Groq
client=Groq(api_key=api_key)

# Defining schema
schema={
    "type":"object",
    "properties":{
        "name":{"type":["string","null"],"description": "Extract ONLY if the user explicitly provides their name. Do NOT infer from email or context."},
        "email":{"type":["string","null"],"format":"email"},
        "phone":{"type":["string","null"]},
        "location":{"type":["string","null"]},
        "age":{"type":["integer","null"],"minimum":0,"maximum":120},
    }
}

function_spec={
    "name": "extract_user_information",
    "description": "Extract name, email, phone, location and age from a chat message.",
    "parameters": schema
}

def extraction(chat_text:str):
  response=client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {"role":"system","content":"You are a information extractor. Your goal is to extract information from the text and return it in structured json output.Do not add anything extra"},
          {"role":"user","content":chat_text}
      ],
      functions=[function_spec],
      function_call={"name":"extract_user_information"}
  )

  raw_args=response.choices[0].message.function_call.arguments
  extracted=json.loads(raw_args)
  return extracted

def validator(extracted:dict):
  validator=Draft7Validator(schema)
  errors=sorted(validator.iter_errors(extracted), key=lambda e: e.path)
  if not errors:
    return True, []
  formatted_errors = []
  for err in errors:
    field_path = ".".join([str(p) for p in err.path]) or "root"
    formatted_errors.append(f"Field '{field_path}': {err.message}")
  return False, formatted_errors



# Testing and demo
sample_chats1 = [
    "Hello I am Ansh and I am student right now in Delhi. I am 21 years old and using anshmaliktest@example.com as my primary email address.",
    "Hi, my name is Rohit Sharma. I'm 28 years old and live in Bengaluru. You can reach me at rohit.sharma@example.com or +91 98765 43210.",
    "Hello. I moved to Mumbai last year. Contact: priya_conn@domain.co. Age: 32."

]

results=[]

for chat in sample_chats1:
    extracted = extraction(chat)
    valid, errors = validator(extracted)
    results.append({
        "chat": chat,
        "extracted": extracted,
        "valid": valid,
        "errors": errors
    })


import pprint
pprint.pprint(results)

[{'chat': 'Hello I am Ansh and I am student right now in Delhi. I am 21 years '
          'old and using anshmaliktest@example.com as my primary email '
          'address.',
  'errors': [],
  'extracted': {'age': 21,
                'email': 'anshmaliktest@example.com',
                'location': 'Delhi',
                'name': 'Ansh',
                'phone': None},
  'valid': True},
 {'chat': "Hi, my name is Rohit Sharma. I'm 28 years old and live in "
          'Bengaluru. You can reach me at rohit.sharma@example.com or +91 '
          '98765 43210.',
  'errors': [],
  'extracted': {'age': 28,
                'email': 'rohit.sharma@example.com',
                'location': 'Bengaluru',
                'name': 'Rohit Sharma',
                'phone': '+91 98765 43210'},
  'valid': True},
 {'chat': 'Hello. I moved to Mumbai last year. Contact: priya_conn@domain.co. '
          'Age: 32.',
  'errors': [],
  'extracted': {'age': 32,
                'email': 'priya_conn@domain.co',
  

In [56]:
from groq._exceptions import BadRequestError
sample_chats2=[
    "Hi, my name is Rohit Sharma. I'm 150 years old and live in Bengaluru. You can reach me at rohit.sharma@example.com or +91 98765 43210."  # Will give error because validation fails
]

results=[]

for chat in sample_chats2:
  try:
      extracted = extraction(chat)
      valid, errors = validator(extracted)
      results.append({
          "chat": chat,
          "extracted": extracted,
          "valid": valid,
          "errors": [str(e) for e in errors]
      })
  except BadRequestError as e:
      error_info =None
      try:
            error_info = e.response.json()  # Convert Response -> dict
      except Exception:
            pass
      if error_info is not None and "error" in error_info:
          msg = error_info["error"].get("message", "Unknown error")
          failed = error_info["error"].get("failed_generation", "")
          results.append({
              "chat": chat,
              "error_message": msg,
              "failed_generation": failed
          })
      else:
          results.append({
              "chat": chat,
              "error_message": str(e)
          })


import pprint
pprint.pprint(results)

[{'chat': "Hi, my name is Rohit Sharma. I'm 150 years old and live in "
          'Bengaluru. You can reach me at rohit.sharma@example.com or +91 '
          '98765 43210.',
  'error_message': 'tool call validation failed: parameters for tool '
                   'extract_user_information did not match schema: errors: '
                   '[`/age`: must be <= 120 but found 150]',
  'failed_generation': '<function=extract_user_information> {"age": 150, '
                       '"name": "Rohit Sharma", "location": "Bengaluru", '
                       '"phone": "+91 98765 43210", "email": '
                       '"rohit.sharma@example.com"} </function>'}]
